<a href="https://www.kaggle.com/code/hachichaeya/test-darija-tts?scriptVersionId=140703431" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install translate

In [2]:
# needed libraries 

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string
import os
from translate import Translator

## Normalizer

In [3]:
# removing puctution
def remove_punctuation(text):
    punctuation = string.punctuation
    text_without_punctuation = ''.join(char for char in text if char not in punctuation)
    return text_without_punctuation


# removing duplicated letters such (example: 'ijaaaaaa' -> 'ija'  'nheeeeebeeek' -> 'nhebek')
def remove_similar_letters(word):
    if not word:
        return word

    i = 0
    result = word[0]

    while i < len(word) - 1:
        if word[i] != word[i + 1]:
            result += word[i + 1]
        i += 1

    return result

## Converting tokens to every possible output

In [4]:
# stemming words

def stemmer(word):
    first_2 =  ['مي', 'مت','ال']
    first_1 = ['ا', 'ي', 'ت',  'ن','م']
    last_1 = ['و', 'ك','ا','ي']
    last_2 = ['نا', 'ني', 'او', 'لو', 'لي', 'لك', 'تو', 'اك''كم', 'يو','هم','وه','يه']
    last_3 = ['وهم','وكم','لنا', 'لكم', 'لهم','يهم','فال']

    removed = False
    # case of negative word (example: 'مايجيكمش')
    if word.startswith('ما') and word.startswith('ش') and len(word)>4 :
        word = word[2:len(word)-1]
    for prefix in first_2:
        if word.startswith(prefix) and len(word)>3:
            word = word[2:]
            removed = True
            break
    if not(removed):
        for prefix in first_1:
            if word.startswith(prefix) and len(word)>4:
                word = word[1:]
                break
    removed = False
    for prefix in last_3:
        if word.endswith(prefix) and len(word)>4:
            word = word[0:len(word)-3]
            removed = True
            break
    if not(removed):
        for prefix in last_2:
            if word.endswith(prefix) and len(word)>3:
                word = word[0:len(word)-2]
                removed = True
                break
    if not(removed):
        for prefix in last_1:
            if word.endswith(prefix) and len(word)>2:
                word = word[0:len(word)-1]
                break
    return word

# function that removes dublicates from array with keeping their initial order (we will use it in next function)
def remove_duplicates_and_keep_order(arr):
    unique_elements = []
    for item in arr:
        if item not in unique_elements:
            unique_elements.append(item)
    return unique_elements


#convert each letter in word with matching letters and return all combinations
def get_combinations(word):
    if (word.startswith('ou') and len(word)>2):
        word = 'ا' + word[2:]
    if (word.startswith('i') or word.startswith('o')):
        word = 'ا' + word[1:]
    if word.endswith('a'):
        word =  word[0:len(word)-1] + 'ا'
    if word.endswith('o'):
        word =  word[0:len(word)-1] + 'و'
    word = word.replace('ch','ش')
    word = word.replace('gh','غ')
    word = word.replace('ou','و')
    word = word.replace('aa','ع')
    word = remove_similar_letters(word)
    combinations = ['']
    latin = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','2','3','5','7','9','8']
    # most common
    arabic = ['ا', 'ب', 'س', 'د', 'ا', 'ف', 'ق', 'ه', 'ي', 'ج', 'ك', 'ل', 'م', 'ن', '', 'ب', 'ق', 'ر', 'س', 'ت', 'و', 'ف', 'و', 'و', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    # less common
    arabic2 = ['ع', 'ب', 'ك', 'د', '', 'ف', 'ق', 'ه', 'ي', 'ج', 'ق', 'ل', 'م', 'ن', '', 'ب', 'ك', 'ر', 'ص', 'ط', 'و', 'ف', 'و', 'و', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    # less common
    arabic3 = ['', 'ب', 'س', 'د', 'ي', 'ف', 'ج', 'ح', 'ي', 'ج', 'ق', 'ل', 'م', 'ن', '', 'ب', 'ك', 'ر', 'س', 'ت', '', 'ف', 'و', '', 'ي', 'ز','أ','ع','خ','ح','ق','غ']
    for i in word:
        temp_combinations = []
        if not(i.lower() in latin):
            for combination in combinations:
                temp_combinations.append(combination + i)
            combinations = remove_duplicates_and_keep_order(temp_combinations)
            continue
        
        letter_index = latin.index(i.lower())
        for combination in combinations:
            temp_combinations.append(remove_similar_letters(combination + arabic[letter_index]))
            temp_combinations.append(remove_similar_letters(combination + arabic2[letter_index]))
            temp_combinations.append(remove_similar_letters(combination + arabic3[letter_index]))
        combinations = remove_duplicates_and_keep_order(temp_combinations)
        skip_next = False
        for combination in combinations:
            if skip_next:
                skip_next = False
                continue
            # other possible combinations
            if combination.find('ته')>-1:
                combinations.insert(combinations.index(combination),combination.replace('ته','ث'))
                skip_next = True
            if combination.find('كه')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('كه','خ'))
            if combination.find('ده')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('ده','ض'))
            if combination.find('ده')>-1:
                combinations.insert(combinations.index(combination)+1,combination.replace('ده','ذ'))
    return remove_duplicates_and_keep_order(combinations)



## Convert sentence to 

In [5]:
#check if the word is all in arabic letters
def is_arabic(word):
    arabic_letters = []
    start = ord('\u0600')  # Start of Arabic Unicode range
    end = ord('\u06FF')  # End of Arabic Unicode range
    for codepoint in range(start, end + 1):
        character = chr(codepoint)
        arabic_letters.append(character)
    for i in word:
        if not(i in arabic_letters):
            return False
    return True


# check if word exists in the vocabulary file

file_path = r'/kaggle/input/test-darija/dictionary.txt'
def exist(combinations):
    existing=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            for combination in combinations:
                if combination == line.strip('\n'):
                    existing.insert(combinations.index(combination),combination)
                if stemmer(combination) in line.strip('\n'):
                    existing.append(combination)
        existing = remove_duplicates_and_keep_order(existing)
    return existing

# translation function
def translate_text(text, from_lang ,target_language):
    translator = Translator(from_lang=from_lang , to_lang=target_language )
    translation = translator.translate(text)
    return translation.split(',')[0]


# convert to arab
def convert_to_arab(post):
    new_post=[]
    french_special_letters = ['é', 'è', 'à', 'ù', 'â', 'ê', 'î', 'ô', 'û', 'ç']
    post = post.replace(' el ',' ال')
    post = post.replace(' l ',' ل')
    post = post.replace(' f ',' في ')
    post = remove_punctuation(post)
    for word in post.split(' '):
        word = word.lower()
        skip = False
        for letter in french_special_letters:
            # If the word is oviously french
            if letter in word.lower():
                new_post.append(translate_text(word,'FR','AR'))
                skip = True
                break
        if skip:
            continue
        if word.isdigit():
            new_post.append(word)
        elif is_arabic(word):
            new_post.append(word)
        elif ('v'in word or 'x' in word or 'p' in word):
            if is_arabic(translate_text(word,'FR','AR').split(' ')[0]):
                new_post.append(translate_text(word,'FR','AR'))
            elif is_arabic(translate_text(word,'EN','AR').split(' ')[0]):
                new_post.append(translate_text(word,'EN','AR'))
            else:
                combinations = get_combinations(word)
                arabic_convertion = exist(combinations)
                if len(arabic_convertion) != 0:
                    new_post.append(arabic_convertion[0])
                else:
                    new_post.append(combinations[0])
                
        else:
            combinations = get_combinations(word)
            arabic_convertion = exist(combinations)
            if len(arabic_convertion) != 0:
                new_post.append(arabic_convertion[0])
            else:
                if is_arabic(translate_text(word,'FR','AR').split(' ')[0]):
                    new_post.append(translate_text(word,'FR','AR'))
                elif is_arabic(translate_text(word,'EN','AR').split(' ')[0]):
                    new_post.append(translate_text(word,'EN','AR'))
                else:
                    new_post.append(combinations[0])
    post = ''
    for word in new_post:
        post = post+' '+ remove_punctuation(word)
    return post[1:]
            


In [6]:
convert_to_arab('nheeeeb internship nshalah hata maa 3atar')

'نحب التدريب الداخلي نسهالاه حتا معا عطار'

In [7]:
convert_to_arab('Aandy chhar nlama7 ama zayed lbhym bhym')

'عندي شهر نلمح اما زيد لبهيم بهيم'

In [8]:
convert_to_arab('mtsirlhech modification')

'متسيرلهاش التغيير'

In [9]:
pip install gTTS

Note: you may need to restart the kernel to use updated packages.


In [10]:
from gtts import gTTS
import os

# Phrase que vous souhaitez convertir en audio en arabe
phrase_arabe = "متسيرلهاش التغيير"

# Créez un objet gTTS avec la phrase en arabe et la langue 'ar'
tts_arabe = gTTS(phrase_arabe, lang='ar')

# Sauvegardez le fichier audio en arabe
tts_arabe.save("output_arabic1.mp3")

# Jouez le fichier audio
os.system("/kaggle/working/output_arabic1.mp3")


sh: 1: /kaggle/working/output_arabic1.mp3: Permission denied


32256